In [ ]:
# Machine Learning Final
# Connor Hehn and John Minogue

import pandas as pd
import numpy as np

In [ ]:
# Reading the data into data frames

df1_E = pd.read_csv("Data/Subject01E.csv")
df1_G = pd.read_csv("Data/Subject02E.csv")

df2_E = pd.read_csv("Data/Subject02E.csv")
df2_G = pd.read_csv("Data/Subject02G.csv")

df3_E = pd.read_csv("Data/Subject03E.csv")
df3_G = pd.read_csv("Data/Subject03G.csv")

df4_E = pd.read_csv("Data/Subject04E.csv")
df4_G = pd.read_csv("Data/Subject04G.csv")

df5_E = pd.read_csv("Data/Subject05E.csv")
df5_G = pd.read_csv("Data/Subject05G.csv")

df6_E = pd.read_csv("Data/Subject06E.csv")
df6_G = pd.read_csv("Data/Subject06G.csv")

df7_E = pd.read_csv("Data/Subject07E.csv")
df7_G = pd.read_csv("Data/Subject07G.csv")

df8_E = pd.read_csv("Data/Subject08E.csv")
df8_G = pd.read_csv("Data/Subject08G.csv")

df9_E = pd.read_csv("Data/Subject09E.csv")
df9_G = pd.read_csv("Data/Subject09G.csv")

df10_E = pd.read_csv("Data/Subject10E.csv")
df10_G = pd.read_csv("Data/Subject10G.csv")
